In [1]:
import psycopg2
import time
import requests
import pandas as pd

In [2]:
# access database
try:
    conn = psycopg2.connect("host=aoe21.cob0rzy4a66o.us-east-1.rds.amazonaws.com dbname=aoe21 user=tico password=1234567890")
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)

In [3]:
# establish cursor
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get curser to the Database")
    print(e)
conn.set_session(autocommit = True)

In [4]:
# wait until time has full tens of minutes
unix_time_stamp = int(time.time())
while (unix_time_stamp % 600) != 0: # 600 for 10 min
    unix_time_stamp = unix_time_stamp + 1

# go back one day to make sure all matches have ended
unix_time_stamp = unix_time_stamp - 86400

# go ahead ten minutes to get the subsequent batch time stamp
subsq_unix_time_stamp = unix_time_stamp + 600
subsq_request = requests.get("https://aoe2.net/api/matches?game=aoe2de&count=1&since=" + str(subsq_unix_time_stamp))
subsq_request = subsq_request.json()

# request
request = requests.get("https://aoe2.net/api/matches?game=aoe2de&count=1000&since=" + str(unix_time_stamp))
request = request.json()

# initialize counter
i = 0

# check if match belongs to current batch
# if yes save data
# if no, all data is collected
while request[i]["match_id"] != subsq_request[0]["match_id"]:
    # match_id
    match_id = request[i]["match_id"]
    # map
    map = request[i]["map_type"]
    # leaderboard_id
    leaderboard_id = request[i]["leaderboard_id"]
    if leaderboard_id != 3:
        i += 1
        continue
    # server
    server = request[i]["server"]
    # started
    started = request[i]["started"]
    # finished
    finished = request[i]["finished"]
    # steam_id_a
    steam_id_a = request[i]["players"][0]["steam_id"]
    # steam_id_b
    steam_id_b = request[i]["players"][1]["steam_id"]
    # country_a
    country_a = request[i]["players"][0]["country"]
    # country_b
    country_b = request[i]["players"][1]["country"]
    # rating_a
    rating_a = request[i]["players"][0]["rating"]
    # rating_b
    rating_b = request[i]["players"][1]["rating"]
    # rating_change_a
    rating_change_a = request[i]["players"][0]["rating_change"]
    # rating_change_b
    rating_change_b = request[i]["players"][1]["rating_change"]
    # civilization_a
    civilization_a = request[i]["players"][0]["civ"]
    # civilization_b
    civilization_b = request[i]["players"][1]["civ"]
    # win_a
    win_a = request[i]["players"][0]["won"]
    # win_b
    win_b = request[i]["players"][1]["won"]
    
    # insert data into table
    cur.execute("INSERT INTO match_history(match_id, map, server, started, finished, steam_id_a, steam_id_b, country_a, country_b, rating_a, rating_b, rating_change_a, rating_change_b, civilization_a, civilization_b, win_a, win_b) VALUES (%s , %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);", (match_id, map, server, started, finished, steam_id_a, steam_id_b, country_a, country_b, rating_a, rating_b, rating_change_a, rating_change_b, civilization_a, civilization_b, win_a, win_b))

    # increase counter
    i += 1